Для зазначених папок (color/, grayscale/, segmented/):

1.Фільтрація шуму (медіанний або Gaussian Blur)

2.Resize до (224×224)

3.Аугментації (обертання, фліпи, зсув, зміна яскравості)

4.Нормалізація пікселів (0…1)

In [ ]:
import cv2
import numpy as np

def preprocess_image(img, target_size=(224,224), augment=True):
    """
    Вхід: img — numpy.ndarray BGR або grayscale
    Повертає: RGB масив float32 нормалізований [0,1]
    """
    # 1. Дenoise
    img = cv2.medianBlur(img, 3)

    # 2. Convert to RGB
    if img.ndim == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # 3. Resize
    img = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)

    # 4. Augmentation (горизонтальний фліп)
    if augment:
        img = cv2.flip(img, 1)

    # 5. Normalize
    img = img.astype("float32") / 255.0
    return img


def preprocess_directory(input_dir, output_dir, target_size=(224,224), augment=True):
   
    import os
    from pathlib import Path
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    for img_name in os.listdir(input_dir):
        in_path = Path(input_dir) / img_name
        out_path = output_dir / img_name

        img = cv2.imread(str(in_path), cv2.IMREAD_UNCHANGED)
        if img is None:
            continue
        proc = preprocess_image(img, target_size, augment)
        # Переводимо назад у uint8 для збереження
        save = (proc * 255).astype(np.uint8)
        # RGB→BGR для cv2.imwrite
        save = cv2.cvtColor(save, cv2.COLOR_RGB2BGR)
        cv2.imwrite(str(out_path), save)

if __name__ == "__main__":
    # preprocess_directory("data/plantvillage/raw/color", "data/processed/color")
    import sys
    src = sys.argv[1]
    dst = sys.argv[2]
    preprocess_directory(src, dst)
    print(f"✅ Preprocessed images from {src} → {dst}")


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np

# Завантажуємо навчену модель
model = load_model("model.h5")

def classify_plant(image, class_labels):
    """
    image: numpy-масив (224×224×3), нормалізований [0,1]
    class_labels: список імен класів у порядку індексів
    """
    # Додаємо батчову розмірність і передбачаємо
    preds = model.predict(np.expand_dims(image, axis=0))[0]
    idx  = np.argmax(preds)
    label      = class_labels[idx]
    confidence = preds[idx]
    return label, confidence

# Приклад виклику
# labels = ["Apple_Black_rot", "Apple_healthy", ...]  
label, conf = classify_plant(processed_img, labels)
print(f"Визначений клас: {label} ({conf*100:.1f}%)")